In [1]:
import os
import json
import argparse
import torch
import torch.nn as nn
import numpy as np
import onnxruntime
from diffusion_planner.model.diffusion_planner import Diffusion_Planner
from diffusion_planner.utils.normalizer import StateNormalizer
from diffusion_planner.model.diffusion_planner import OnnxWrapper 

In [2]:
# --- 您的代码：加载配置（这部分是正确的）---
current_directory = os.getcwd()
path = os.path.join(current_directory, "checkpoints", "args.json")

# 1. 加载JSON文件
with open(path, 'r') as f:
    config_dict = json.load(f)

# 2. 将字典转换为Namespace对象
args = argparse.Namespace(**config_dict)
args.guidance_fn = None
print("从JSON加载的原始配置 (args.state_normalizer 类型):", type(args.state_normalizer))


# --- 关键修正步骤：手动实例化对象 ---
# 检查 state_normalizer 是否是一个需要被实例化的字典
if hasattr(args, 'state_normalizer') and isinstance(args.state_normalizer, dict):
    print("正在将 state_normalizer 从字典转换为实例对象...")
    # 从字典中解包参数来创建对象
    normalizer_params = args.state_normalizer
    state_normalizer_object = StateNormalizer(**normalizer_params)
    
    # 将 args 中的字典替换为真正的实例对象
    args.state_normalizer = state_normalizer_object
    print("修正完成! 当前 config.state_normalizer 类型:", type(args.state_normalizer))
# --- 修正结束 ---

# --- 现在，使用修正后的 args 初始化模型 ---
print("正在初始化 Diffusion_Planner 模型...")
model = Diffusion_Planner(args)
print("模型初始化完成!")

从JSON加载的原始配置 (args.state_normalizer 类型): <class 'dict'>
正在将 state_normalizer 从字典转换为实例对象...
修正完成! 当前 config.state_normalizer 类型: <class 'diffusion_planner.utils.normalizer.StateNormalizer'>
正在初始化 Diffusion_Planner 模型...
模型初始化完成!


In [3]:
# 导入 OrderedDict
from collections import OrderedDict

# 1. 加载检查点文件
checkpoint_path = "./checkpoints/model.pth"
print(f"正在从 {checkpoint_path} 加载权重...")
original_state_dict = torch.load(checkpoint_path, map_location='cpu')['model']

# 2. 只处理 'module.' 前缀 (如果需要)
new_state_dict = OrderedDict()
for k, v in original_state_dict.items():
    name = k[7:] if k.startswith('module.') else k
    new_state_dict[name] = v

# 3. 直接加载，不再需要重命名 'in_proj_weight'
model.load_state_dict(new_state_dict)
print("权重已成功加载！")

正在从 ./checkpoints/model.pth 加载权重...
权重已成功加载！


In [4]:
# --- 2d. 设置评估模式并包装模型 ---
model.eval()
wrapped_model = OnnxWrapper(model)
wrapped_model.eval()
print("模型已设置为评估模式并已包装。")

模型已设置为评估模式并已包装。


In [5]:
# ==============================================================================
# 步骤 3: 准备导出所需的“三件套”
# ==============================================================================
print("\n--- 步骤 3: 准备导出参数 ---")

# --- 3a. 伪输入元组 (Dummy Inputs) ---
dummy_neighbor_agents_past = torch.randn(1, 32, 21, 11)
dummy_ego_current_state = torch.randn(1, 10)
dummy_static_objects = torch.randn(1, 5, 10)
dummy_lanes = torch.randn(1, 70, 20, 12)
dummy_lanes_speed_limit = torch.randn(1, 70, 1)
dummy_lanes_has_speed_limit = torch.ones(1, 70, 1).bool() # 已修正
dummy_route_lanes = torch.randn(1, 25, 20, 12)
dummy_route_lanes_speed_limit = torch.randn(1, 25, 1)
dummy_route_lanes_has_speed_limit = torch.ones(1, 25, 1).bool() # 已修正

dummy_inputs_tuple = (
    dummy_neighbor_agents_past, dummy_ego_current_state, dummy_static_objects,
    dummy_lanes, dummy_lanes_speed_limit, dummy_lanes_has_speed_limit,
    dummy_route_lanes, dummy_route_lanes_speed_limit, dummy_route_lanes_has_speed_limit,
)


--- 步骤 3: 准备导出参数 ---


In [6]:
# --- 3b. 输入输出节点名称 ---
input_names = [
    "neighbor_agents_past", "ego_current_state", "static_objects", "lanes",
    "lanes_speed_limit", "lanes_has_speed_limit", "route_lanes",
    "route_lanes_speed_limit", "route_lanes_has_speed_limit",
]
# 推理时模型返回 {"prediction": x0}，所以输出节点名为 "prediction"
output_names = ["prediction"] 

# --- 3c. 动态轴 ---
dynamic_axes = {name: {0: "batch_size"} for name in input_names}
dynamic_axes[output_names[0]] = {0: "batch_size"}
print("导出参数准备就绪。")

导出参数准备就绪。


In [7]:
# ==============================================================================
# 步骤 4: 执行导出并验证
# ==============================================================================
print("\n--- 步骤 4: 执行导出与验证 ---")
onnx_filename = "diffusion_planner.onnx"
print(f"即将导出 ONNX 模型到: {onnx_filename}")

try:
    torch.onnx.export(
        wrapped_model,
        dummy_inputs_tuple,
        onnx_filename,
        input_names=input_names,
        output_names=output_names,
        dynamic_axes=dynamic_axes,
        opset_version=11,
        verbose=False
    )
    print(f"\n✅ ONNX 模型导出成功!")


except Exception as e:
    print(f"\n❌ 导出过程中发生错误: {e}")
    import traceback
    traceback.print_exc()


--- 步骤 4: 执行导出与验证 ---
即将导出 ONNX 模型到: diffusion_planner.onnx


/home/bydguikong/yy_ws/Diffusion-Planner/diffusion_planner/model/module/encoder.py:176: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if valid_indices.sum() > 0:
/home/bydguikong/yy_ws/Diffusion-Planner/diffusion_planner/model/module/encoder.py:249: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if has_speed_limit.sum() > 0:
/home/bydguikong/yy_ws/Diffusion-Planner/diffusion_planner/model/module/encoder.py:253: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so

============= Diagnostic Run torch.onnx.export version 2.0.0+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================


✅ ONNX 模型导出成功!


In [8]:
# --- （强烈推荐）步骤4：验证导出的模型 (已修正) ---
import onnxruntime
import numpy as np

print("\n正在验证导出的 ONNX 模型...")

# 1. 创建 ONNX Runtime 推理会话
ort_session = onnxruntime.InferenceSession(onnx_filename)

# 2. 准备一个从输入名到伪输入张量的查找字典
#    (input_names 和 dummy_inputs_tuple 是我们之前定义的)
dummy_inputs_by_name = dict(zip(input_names, dummy_inputs_tuple))

# 3. 获取 ONNX 模型真正需要的输入节点的名称
actual_input_names = [inp.name for inp in ort_session.get_inputs()]
print(f"   - ONNX 模型实际有 {len(actual_input_names)} 个输入: {actual_input_names}")

# 辅助函数，将 torch tensor 转为 numpy array
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# 4. 根据模型实际需要的输入，来构建 onnxruntime 的输入字典
ort_inputs = {
    name: to_numpy(dummy_inputs_by_name[name])
    for name in actual_input_names
}

# 5. 执行推理
ort_outputs = ort_session.run(None, ort_inputs)

# 6. 打印结果，检查形状
print("✅ ONNX 模型验证成功!")
print(f"   - ONNX Runtime 推理输出数量: {len(ort_outputs)}")
# 假设您的 Encoder 输出是一个字典，其中包含一个名为 'encoding' 的键
# ONNX 导出后，这个键名会成为输出节点名
output_node_names = [out.name for out in ort_session.get_outputs()]
print(f"   - ONNX 输出节点名称: {output_node_names}")
print(f"   - 第一个输出 '{output_node_names[0]}' 的形状: {ort_outputs[0].shape}")



正在验证导出的 ONNX 模型...
   - ONNX 模型实际有 7 个输入: ['neighbor_agents_past', 'ego_current_state', 'static_objects', 'lanes', 'lanes_speed_limit', 'lanes_has_speed_limit', 'route_lanes']
✅ ONNX 模型验证成功!
   - ONNX Runtime 推理输出数量: 1
   - ONNX 输出节点名称: ['prediction']
   - 第一个输出 'prediction' 的形状: (1, 11, 80, 4)


In [11]:
!python -m onnxsim ./diffusion_planner.onnx ./simplified_model.onnx

Your model contains "Tile" ops or/and "ConstantOfShape" ops. Folding these ops 
can make the simplified model much larger. If it is not expected, please specify
"--no-large-tensor" (which will lose some optimization chances)
Simplifying...
Finish! Here is the difference:
┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃                  ┃ Original Model ┃ Simplified Model ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ Add              │ 1351           │ 1338             │
│ And              │ 74             │ 74               │
│ Atan             │ 1              │ 1                │
│ Cast             │ 356            │ 82               │
│ Concat           │ 486            │ 302              │
│ Constant         │ 3909           │ 440              │
│ ConstantOfShape  │ 165            │ 26               │
│ Cos              │ 12             │ 12               │
│ Div              │ 396            │ 396              │
│ Equal            │ 146            │ 80    

In [12]:
import netron
current_directory = os.getcwd()
netron.start(current_directory + "/diffusion_planner.onnx")
netron.start(current_directory + "/simplified_model.onnx")


Serving '/home/bydguikong/yy_ws/Diffusion-Planner/diffusion_planner.onnx' at http://localhost:8080
Serving '/home/bydguikong/yy_ws/Diffusion-Planner/simplified_model.onnx' at http://localhost:8081


('localhost', 8081)